<a href="https://colab.research.google.com/github/sainijagjit/CS6910-A2/blob/main/Part%20B/Assignment_2_iNaturalist_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
! pip install pytorch-lightning --quiet
# install weights and biases
# !pip uninstall wandb -y
!pip install wandb --quiet
# !pip install --upgrade wandb

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as L
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler, Subset
from sklearn.model_selection import train_test_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import os
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.models as models
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import EarlyStopping,Callback,ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
from google.colab import userdata

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
zip_path = "/content/drive/MyDrive/Deep Learning/nature_12K.zip"
!cp "{zip_path}" .
!unzip -q nature_12K.zip
!rm nature_12K.zip

In [9]:
wandb.login(key=userdata.get('wandb_key'))
wandb_project="CS6910 - Assignment 2"
wandb_entity="ns24z060"

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


**Loading & Splitting Dataset**

In [10]:
class DataModule(L.LightningDataModule):
    def __init__(self, batch_size=64, augment_data= True ,data_dir = './inaturalist_12K',num_workers=2):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size
        self.augment_data = augment_data
        self.num_workers=num_workers


        self.augmentation = transforms.Compose([
              transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

        self.transform = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])



    def prepare_data(self):
        self.train_dataset = ImageFolder(root=f'{self.data_dir}/train')
        self.test_dataset = ImageFolder(root=f'{self.data_dir}/val')
        print(f'Classes {self.train_dataset.classes}')
        print(f'Test Samples {len(self.test_dataset)}')


    def setup(self, stage=None):
        self.test_dataset.transform = self.transform
        labels = self.train_dataset.targets
        train_indices, val_indices = train_test_split(
            range(len(self.train_dataset)),
            test_size=0.2,
            random_state=42,
            stratify=labels
        )
        if stage=='fit':
          class_names = self.train_dataset.classes
          class_counts = {class_name: 0 for class_name in class_names}
          for val_indice in val_indices:
              _, class_idx = self.train_dataset[val_indice]
              class_name = class_names[class_idx]
              class_counts[class_name] += 1
          print('Validation Datased Class Distribution')
          for class_name, count in class_counts.items():
              print(f"Class: {class_name}, No. of samples: {count}")
        self.val_dataset = Subset(self.train_dataset, val_indices)
        self.train_dataset = Subset(self.train_dataset, train_indices)
        if stage=='fit':
          print(f'Train Samples {len(self.train_dataset)}')
          print(f'Val Samples {len(self.val_dataset)}')

        # num_classes = len(class_names)
        # input_dims = self.test_dataset[0][0].shape
        # print(f'Input image shape ({input_dims[0]},{input_dims[1]},{input_dims[2]})')
        # print('-------------')


        if self.augment_data:
          self.train_dataset.dataset.transform = self.augmentation
        self.val_dataset.dataset.transform = self.transform

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)


    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=self.num_workers)


    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=self.num_workers)



In [11]:
class Net(nn.Module):
  def __init__(self):
      super().__init__()
      self.model = models.resnet50(weights='IMAGENET1K_V1')
      num_features = self.model.fc.in_features
      self.model.fc = nn.Linear(num_features, 10)

  def forward(self, x):
        x =  self.model(x)
        return x

In [12]:
class MyCnnModel(L.LightningModule):
    def __init__(self,
                 learning_rate=0.01,
                 loss_function='cross_entropy',
                 optimizer='Adam'):
        super().__init__()
        self.save_hyperparameters()
        self.model = Net()
        self.accuracy = Accuracy(task="multiclass", num_classes=10)
        self.loss_function = getattr(F, loss_function)
        self.learning_rate = learning_rate
        self.optimizer = optimizer

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
      loss,acc = self._common_step(batch,batch_idx)
      self.log('train_loss', loss, on_step=False, on_epoch=True, logger=True)
      self.log('train_acc', acc, on_step=False, on_epoch=True, logger=True)
      return loss

    def validation_step(self, batch, batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('val_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('val_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss


    def test_step(self,batch,batch_idx):
      loss,acc =  self._common_step(batch,batch_idx)
      self.log('test_loss', loss, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      self.log('test_acc', acc, prog_bar=True, on_step=False, on_epoch=True, logger=True)
      return loss

    def _common_step(self, batch, batch_idx):
      inputs, target = batch
      output = self.forward(inputs)
      loss = self.loss_function(output, target)
      acc = self.accuracy(output, target)
      return loss,acc

    def configure_optimizers(self):
      return getattr(torch.optim,self.optimizer)(self.parameters(), lr=self.learning_rate)

In [13]:
class ImagePredictionLogger(L.callbacks.Callback):
    def __init__(self, samples, num_samples=30):
        super().__init__()
        self.num_samples = num_samples
        self.sample_imgs, self.val_labels = samples
        self.classes = classes = ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']

    def on_validation_epoch_end(self, trainer, pl_module):
        sample_imgs = self.sample_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        logits = pl_module(sample_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "examples":[wandb.Image(x, caption=f"Pred:{self.classes[pred]}, Label:{self.classes[y]}")
                           for x, pred, y in zip(sample_imgs[:self.num_samples],
                                                 preds[:self.num_samples],
                                                 val_labels[:self.num_samples])]
            })

    def on_test_epoch_end(self, trainer, pl_module):
        sample_imgs = self.sample_imgs.to(device=pl_module.device)
        val_labels = self.val_labels.to(device=pl_module.device)
        logits = pl_module(sample_imgs)
        preds = torch.argmax(logits, -1)

        trainer.logger.experiment.log({
            "Test Predictions":[wandb.Image(x, caption=f"Pred:{self.classes[pred]}, Label:{self.classes[y]}")
                           for x, pred, y in zip(sample_imgs[:self.num_samples],
                                                 preds[:self.num_samples],
                                                 val_labels[:self.num_samples])]
            })


In [14]:
def train():

    config_defaults = {
        "augment_data": True,
        "learning_rate": 1e-3,
        "activation_function": 'ReLU',
        "batch_size": 32,
        "epochs": 10
    }

    wandb.init(config=config_defaults, magic=True)
    config = wandb.config

    name = "_".join(["lr", str(config.learning_rate), "augmt", str(config.augment_data),"batch",str(config.batch_size),"activ",config.activation_function,"eph",str(config.epochs)])
    api = wandb.Api()
    runs = api.runs(wandb.run.project, {"$and": [{"config.name": name}]})
    if len(runs) > 0:
        return
    wandb.run.name = name
    dm = DataModule(config.batch_size,config.augment_data)
    dm.prepare_data()
    dm.setup()
    val_samples = next(iter(dm.val_dataloader()))
    model = MyCnnModel(config.learning_rate)
    checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode='max',dirpath='/content/drive/MyDrive/Deep Learning/Part 2',save_top_k=1,filename='best_model')
    wandb_logger = WandbLogger(project=wandb_project, job_type='train', log_model='all')
    early_stop_callback = EarlyStopping(monitor="val_loss",patience=3)
    trainer = L.Trainer(
      max_epochs=config.epochs,
      check_val_every_n_epoch=1,
      log_every_n_steps=1,
      logger=wandb_logger,
      callbacks=[checkpoint_callback,ImagePredictionLogger(val_samples),early_stop_callback])
    trainer.fit(model, dm)
    best_model_path = checkpoint_callback.best_model_path
    wandb.log({f'best_model_path{wandb.run.name}': best_model_path})

In [ ]:
sweep_config = {
  "name": "Assignment_2_iNaturalis",
  "method": "bayes",
  'metric': {
      'name': 'val_acc',
      'goal': 'maximize'
    },
  "parameters": {
        "augment_data":{
          "values": [True,False]
        },
        "learning_rate": {
            "values": [1e-3,1e-4,1e-5]
        },
        "batch_size": {
            "values": [8,32,64,128]
        },
        "epochs": {
            "values": [10,15,20]
        }
    }
}


# sweep_id = wandb.sweep(sweep_config, entity=wandb_entity, project=wandb_project)
wandb.agent("4pmkvg7a", train, count=50, entity=wandb_entity, project=wandb_project)
wandb.finish()

wandb: Agent Starting Run: t4wzzat1 with config:
wandb: 	activation_function: ReLU
wandb: 	augment_data: True
wandb: 	batch_size: 128
wandb: 	dense_neurons: 128
wandb: 	epochs: 20
wandb: 	learning_rate: 0.0001
wandb: Currently logged in as: ns24z060. Use `wandb login --relogin` to force relogin
wandb: WARNING wandb.init() arguments ignored because wandb magic has already been initialized


Problem at: <ipython-input-10-3de5d64c344e> 11 train


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 1181, in init
    run = wi.init()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 841, in init
    run._on_start()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2398, in _on_start
    self._on_ready()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2435, in _on_ready
    self._register_telemetry_import_hooks()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2427, in _register_telemetry_import_hooks
    register_post_import_hook(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/import_hooks.py", line 80, in register_post_import_hook
    hook(module)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2418, in _telemetry_import_hook
    name = getattr(module, "__name__", None)
  File "/usr/local/lib/python3.10/dist-p

Run t4wzzat1 errored:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 1181, in init
    run = wi.init()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_init.py", line 841, in init
    run._on_start()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2398, in _on_start
    self._on_ready()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2435, in _on_ready
    self._register_telemetry_import_hooks()
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2427, in _register_telemetry_import_hooks
    register_post_import_hook(
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/lib/import_hooks.py", line 80, in register_post_import_hook
    hook(module)
  File "/usr/local/lib/python3.10/dist-packages/wandb/sdk/wandb_run.py", line 2418, in _telemetry_import_hook
    name = getattr(module, "__name__", None)
  File "/usr/local

Classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Test Samples 2000


...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


epoch,▁▁██
train_acc,▁█
train_loss,█▁
trainer/global_step,▁▁██
val_acc,▁█
val_loss,█▁
epoch,1
train_acc,0.74609
train_loss,1.16301
trainer/global_step,125
val_acc,0.7735


Load Best Model & Check Test Accuracy

In [15]:
model = MyCnnModel.load_from_checkpoint(checkpoint_path='/content/drive/MyDrive/Deep Learning/Part 2/best_model.ckpt')
dm = DataModule()
dm.prepare_data()
dm.setup()
test_samples = next(iter(dm.test_dataloader()))
wandb_logger = WandbLogger(project=wandb_project, job_type='test', name='test_images_prediction')
trainer = L.Trainer(logger=wandb_logger,callbacks=[ImagePredictionLogger(test_samples)])
trainer.test(model,dm)
wandb.finish()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 86.8MB/s]


Classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Test Samples 2000


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: Currently logged in as: ns24z060. Use `wandb login --relogin` to force relogin


Classes ['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']
Test Samples 2000


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8305000066757202     │
│         test_loss         │    0.6203917860984802     │
└───────────────────────────┴───────────────────────────┘

epoch,▁
test_acc,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_acc,0.8305
test_loss,0.62039
trainer/global_step,0
